In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras as K
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('100k_dataset.csv', delimiter=' ')

In [3]:
df.head()

,depth,prev_room,prev_exit,room,exit_0,exit_1,exit_2,exit_3,exit_4,exit_portal
0,0,Start,0,FiveWayCrossing,Corridor,LeftTurn,None,Corridor,Stairs,0
1,0,Start,0,FiveWayCrossing,Corridor,RightTurn,None,Corridor,RightTurn,0
2,2,RightTurn,0,Corridor,SquareRoom,None,Corridor,None,None,0
3,0,Start,0,FiveWayCrossing,Stairs,None,None,None,Corridor,4
4,1,FiveWayCrossing,4,Corridor,Corridor,SpiralStaircase,Stairs,None,None,0


In [4]:
# one hot
for col in [
        'prev_room',
        'prev_exit',
        'room',
        'exit_0',
        'exit_1',
        'exit_2',
        'exit_3',
        'exit_4']:
    df = pd.concat([df, pd.get_dummies(df[col], prefix=col)], axis=1)
    df.drop(col, axis=1, inplace=True)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
        df.drop('exit_portal', axis=1),
        df['exit_portal'],
        test_size=0.1,
        random_state=1337)

In [6]:
print("X_train:", X_train.shape)
print("y_train:", y_train.shape)
print("X_test:", X_test.shape)
print("y_test:", y_test.shape)

X_train: (219037, 79)
y_train: (219037,)
X_test: (24338, 79)
y_test: (24338,)


In [7]:
model = K.Sequential()
model.add(K.layers.InputLayer(input_shape=(X_train.shape[1],)))
model.add(K.layers.Dense(128, activation='relu'))
model.add(K.layers.Dense(64, activation='relu'))
model.add(K.layers.Dense(5, activation='softmax'))
model.compile(
        optimizer='adam',
        loss=K.losses.SparseCategoricalCrossentropy(),
        metrics=['accuracy'])

In [8]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
6845/6845 [==============================] - 4s 488us/step - loss: 0.4169 - accuracy: 0.7890
Epoch 2/10
6845/6845 [==============================] - 3s 497us/step - loss: 0.3992 - accuracy: 0.7960
Epoch 3/10
6845/6845 [==============================] - 3s 498us/step - loss: 0.3966 - accuracy: 0.7967
Epoch 4/10
6845/6845 [==============================] - 3s 492us/step - loss: 0.3950 - accuracy: 0.7976
Epoch 5/10
6845/6845 [==============================] - 3s 506us/step - loss: 0.3935 - accuracy: 0.7983
Epoch 6/10
6845/6845 [==============================] - 3s 485us/step - loss: 0.3924 - accuracy: 0.7996
Epoch 7/10
6845/6845 [==============================] - 3s 493us/step - loss: 0.3917 - accuracy: 0.8001
Epoch 8/10
6845/6845 [==============================] - 3s 505us/step - loss: 0.3904 - accuracy: 0.8007
Epoch 9/10
6845/6845 [==============================] - 3s 501us/step - loss: 0.3897 - accuracy: 0.8014
Epoch 10/10
6845/6845 [==============================] - 3s 482u

In [9]:
model.save("shnet_1.keras")

In [10]:
results = model.evaluate(X_test, y_test)

761/761 [==============================] - 0s 304us/step - loss: 0.4040 - accuracy: 0.7946
